<a href="https://colab.research.google.com/github/kdj0712/study_data_analytics/blob/main/docs/quests/CDAs/CDA_categorical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## X(범주형) / Y(범주형) 케이스


### 사전 작업

In [ ]:
# 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [1]:
import pandas as pd
import scipy
import scipy.stats as stats
import seaborn as sns
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import matplotlib.font_manager as fm
# 설치된 폰트 중 'NanumGothic' 찾기
for font in fm.fontManager.ttflist:
    if 'NanumGothic' in font.name:
        plt.rcParams['font.family'] = font.name
        break
plt.rcParams['axes.unicode_minus'] = False
import numpy as np

In [2]:
df_fruitstores = pd.read_csv('/content/drive/MyDrive/data_analytics/CSV파일/fruitstore_saleshistory.csv')
df_fruitstores[:2]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05


## 요일(X)이 가장 인기 있는 과일 카테고리 판매(Y)에 영향을 미칩니까?
- 분석 방법 : 카이제곱 검정 또는 기대 셀 빈도가 5 미만인 경우 피셔의 정확 검정을 사용하여 특정 요일에 과일 카테고리의 인기에 통계적으로 유의미한 차이가 있는지 평가합니다.


### 분석 내용 : 특정 요일에 과일 카테고리의 인기에 통계적으로 유의미한 차이가 있는가?
- 귀무가설 : 특정 요일에 따라 과일 카테고리의 판매가 달라지지 않는다. (p-value >= 0.05)
- 대립가설 : 특정 요일에 따라 과일 카테고리의 판매가 달라
진다.(p-value <= 0.05)

In [3]:
# 로딩 한 데이터 중 판매 품목이 과일인 항목만 추출하여 별도의 데이터 프레임 생성
df_quest1 = df_fruitstores
conditions_1 = '물품중분류 == "과일"'
df_quest2 = df_quest1.query(conditions_1)
df_quest2[:2]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
62,2018-01-02,화,1,1,445131904,정상회원,과실,과일,건대추,건대추(500g),1.0,수지구,풍덕천동,여,58.0,50대,22973,매장C,NaN,12:18


In [4]:
# 과일 이외의 판매 품목 별 요일 별 판매 건수를 분류한 내용을 피못 테이블 기능으로 새로 생성하여 df_quest4라는 데이터 프레임에 저장
df_fruitstores4 = df_fruitstores
df_quest4 = df_fruitstores4 .pivot_table(index= '물품중분류',columns='요일', aggfunc='size')
df_quest4

요일,금,목,수,월,일,토,화
물품중분류,,,,,,,
가루,1779.0,1801.0,1969.0,2294.0,1051.0,1663.0,2034.0
감자,936.0,863.0,842.0,890.0,294.0,886.0,900.0
건강일반,866.0,856.0,914.0,1106.0,530.0,806.0,1095.0
건어물,1405.0,1378.0,1523.0,1755.0,781.0,1268.0,1544.0
견과,637.0,650.0,680.0,581.0,255.0,432.0,543.0
...,...,...,...,...,...,...,...
콩/화본/깨,954.0,1116.0,1002.0,1169.0,547.0,962.0,1139.0
콩나물,2957.0,2824.0,3080.0,3365.0,1313.0,2246.0,3040.0
해조,1933.0,1838.0,1925.0,2403.0,971.0,1826.0,2132.0


In [5]:
# df_fruitstores 데이터프레임에서 ['물품중분류']컬럼에 있는 항목에서 과일인 항목을 찾아 과일은 '과일' record에 아닌 항목은 '과일 아님'record에 저장한다.
df_fruitstores4['과일여부'] = np.where(df_fruitstores4['물품중분류'] == '과일', '과일', '과일 아님')

In [6]:
# df_4 record에서 '과일 여부'항목을 인덱스로 하여, 요일별로 판매 건수를 추출한 뒤 피봇 테이블을 통해 df_quest5 데이터 프레임에 저장한다.
df_quest5 = df_fruitstores4.pivot_table(index='과일여부', columns='요일', aggfunc='size')

In [7]:
# 데이터 프레임 확인
df_quest5

요일,금,목,수,월,일,토,화
과일여부,,,,,,,
과일,2552,2321,2213,2586,1169,2347,2214
과일 아님,107319,101823,102777,121216,52908,95144,108982


In [8]:
# 카이제곱을 통해 두 종류 간의 P-value 확인을 통한 접점 확인
stats.chi2_contingency(observed=df_quest5,correction=True)

Chi2ContingencyResult(statistic=61.271132941778866, pvalue=2.4825470812456557e-11, dof=6, expected_freq=array([[  2398.38817355,   2273.37275483,   2291.84019751,
          2702.48976219,   1180.45377999,   2128.14356316,
          2427.31176877],
       [107472.61182645, 101870.62724517, 102698.15980249,
        121099.51023781,  52896.54622001,  95362.85643684,
        108768.68823123]]))

In [9]:
from scipy.stats import chi2_contingency
# 데이터 프레임 생성
data = {'금': [2552, 107319], '목': [2321, 101823], '수': [2213, 102777], '월': [2586, 121216], '일': [1169, 52908], '토': [2347, 95144], '화': [2214, 108982]}
df = pd.DataFrame(data, index=['과일', '과일 아님'])

# 카이제곱 검정 실시
chi2, p, dof, expected = chi2_contingency(df)

# 크래머의 V 계산
n = np.sum(df.sum()) # 총 관측치 수
min_dim = min(df.shape)-1 # 열 또는 행의 수에서 1을 뺀 값 중 작은 값

V = np.sqrt((chi2/n) / min_dim)
print('크래머의 V:', V)

크래머의 V: 0.009318748571356375


In [10]:
# P-value 정제
round(2.4825470812456557e-11,6)

0.0

## 분석 결과 : 특정 요일에 과일 카테고리의 인기에 통계적으로 유의미한 차이가 있는가?
- 통계 결론 : P-value가 매우 미미하므로 귀무가설을 기각할 근거가 된다.
- 사실 결론 : 특정 요일에 따라 과일 카테고리의 판매가 달라 진다.(p-value <= 0.05) 그러나, 상관 계수(크래머의 V)가 매우 작으므로 그 차이는 매우 미미하다.